In [1]:
from os.path import exists, join, splitext
from urllib.request import urlretrieve

archive_url = (
    'https://egriddata.org/sites/default/files/'
    'GSO_RNM_GIS_Network.zip')
archive_path = '/tmp/greensboro-synthetic-network.zip'
archive_folder = splitext(archive_path)[0]
if not exists(archive_path):
    urlretrieve(archive_url, archive_path)
if not exists(archive_folder):
    archive_folder = uncompress(archive_path)
source_folder = join(archive_folder, 'GSO_RNM_GIS_Network', 'Rural')

In [2]:
ls $source_folder/*.shp

/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/DistribTransf_N.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/DummyEquip.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/HVMVSubstation_N.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Line_N.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_branches.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_nodes.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_nodes.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/NewConsumerGreenfield_N.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/StreetMap_branches.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/StreetMap_nodes.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/SwitchingDevices_N.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Transformer_N.shp
/tmp/greensboro-synthetic-network/GS

In [3]:
import geotable
t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_nodes.shp')
t[:3]

,Node,ConnGroup,InSettlem,ReliabZone,OH,NomV,Vpu,P,Q,Qmin,Qmax,Vpu_obj,Phases,PhasesV,Subest,Feeder,geometry_object,geometry_layer,geometry_proj4
0,ST_MAT,1,x,x,x,230.00,1,26100.3 ;,17178.8 ;,0,0,1,0P,0P_AT,True,True,POINT (625155.6499999999 4018544.7),Network_nodes,+proj=longlat +datum=WGS84 +no_defs
1,RCLV33,1,x,S,x,0.42,1.00118 ;,-0.760008,-0.370004,0,0,0,A,A_BT,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,POINT (622867.4 4016120.6),Network_nodes,+proj=longlat +datum=WGS84 +no_defs
2,RCLV47,1,x,S,x,0.42,0.999787 ;,-0.760008,-0.370004,0,0,0,B,B_BT,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,POINT (622900.3 4016208),Network_nodes,+proj=longlat +datum=WGS84 +no_defs


In [4]:
import geotable
t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_nodes.shp')
t[:3]

,Node,ConnGroup,InSettlem,ReliabZone,OH,NomV,Vpu,P,Q,Qmin,Qmax,Vpu_obj,Phases,PhasesV,Subest,Feeder,geometry_object,geometry_layer,geometry_proj4
0,RCLV33,1,x,S,x,0.42,1.00118 ;,-0.760008,-0.370004,0,0,0,A,A_BT,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,POINT (622867.4 4016120.6),Network_NEW_nodes,+proj=longlat +datum=WGS84 +no_defs
1,RCLV47,1,x,S,x,0.42,0.999787 ;,-0.760008,-0.370004,0,0,0,B,B_BT,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,POINT (622900.3 4016208),Network_NEW_nodes,+proj=longlat +datum=WGS84 +no_defs
2,RCLV37,1,x,S,x,0.42,1.00084 ;,-2.74003,-1.33001,0,0,0,A,A_BT,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,POINT (622858.9 4016228.1),Network_NEW_nodes,+proj=longlat +datum=WGS84 +no_defs


In [5]:
t.iloc[0]

Node                                            RCLV33
ConnGroup                                            1
InSettlem                                            x
ReliabZone                                           S
OH                                                   x
NomV                                              0.42
Vpu                                          1.00118 ;
P                                            -0.760008
Q                                            -0.370004
Qmin                                                 0
Qmax                                                 0
Vpu_obj                                              0
Phases                                               A
PhasesV                                           A_BT
Subest                                  S_nSSEE1_12.47
Feeder                   S_nSSEE1_12.47 -> S_nCCTT2619
geometry_object             POINT (622867.4 4016120.6)
geometry_layer                       Network_NEW_nodes
geometry_p

In [6]:
import geotable
t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_branches.shp')
t[:3]

,Node_A,Node_B,N_Son,N_Parent,P_a,Q_a,P_b,Q_b,P_Flow,Status,...,Y22_pu,NumLines,TimeLoc,TimeCrews,TimeRed,Subest,Feeder,geometry_object,geometry_layer,geometry_proj4
0,S_nCCTT6,S_nCCTT6_BT,S_nCCTT6_BT,S_nCCTT6,13.2614 ;,7.42544 ;,-13.2614 ;,-6.3979 ;,15.1987 <-;,1,...,"(0,-2.08333)",0,0.0,0.0,0.0,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,"LINESTRING (623052.4 4016495.5, 623052.4 40164...",Network_NEW_branches,+proj=longlat +datum=WGS84 +no_defs
1,S_nCCTT4,S_nCCTT4_BT,S_nCCTT4_BT,S_nCCTT4,0.760142 ;,0.373884 ;,-0.760142 ;,-0.370019 ;,0.847116 <-;,1,...,"(0,-1.85185)",0,0.0,0.0,0.0,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,"LINESTRING (623015.2 4015964.1, 623015.2 40159...",Network_NEW_branches,+proj=longlat +datum=WGS84 +no_defs
2,S_nCCTT5,S_nCCTT5_BT,S_nCCTT5_BT,S_nCCTT5,12.6144 ;,7.01527 ;,-12.6144 ;,-6.08198 ;,14.4339 <-;,1,...,"(0,-2.08333)",0,0.0,0.0,0.0,S_nSSEE1_12.47,S_nSSEE1_12.47 -> S_nCCTT2619,"LINESTRING (622895.8 4015965.7, 622895.8 40159...",Network_NEW_branches,+proj=longlat +datum=WGS84 +no_defs


In [7]:
t.iloc[0]

Node_A                                                      S_nCCTT6
Node_B                                                   S_nCCTT6_BT
N_Son                                                    S_nCCTT6_BT
N_Parent                                                    S_nCCTT6
P_a                                                        13.2614 ;
Q_a                                                        7.42544 ;
P_b                                                       -13.2614 ;
Q_b                                                        -6.3979 ;
P_Flow                                                   15.1987 <-;
Status                                                             1
SNom                                                               0
Overload                                                           0
Margin                                                          9.77
Sa_coef                                                            1
Sb_coef                           

In [8]:
from glob import glob
source_paths = glob(join(source_folder, '*.shp'))
source_paths

['/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/VoltageRegulator_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/TransSubstation_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Transformer_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/SwitchingDevices_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/StreetMap_nodes.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/StreetMap_branches.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/NewConsumerGreenfield_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_nodes.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_nodes.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_branches.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Line_N.shp',
 '/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/H

In [9]:
import geotable
t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/TransSubstation_N.shp')
t.iloc[0]

Node                                            S_nST0
Code                                            S_nST0
NomV_kV                                              0
Equip                                                 
Size_kVA                                        300000
E_kWh                                        7.884e+08
Ppeak_kW                                        300000
Qpeak_kVAr                                      300000
MaxNumOut                                           12
InvCOut                                         729000
InvC                                                 0
PMainC                                               0
PMainT                                               0
CMainC                                               0
NmaxTransf                                           2
geometry_object               POINT (622703 4007590.1)
geometry_layer                       TransSubstation_N
geometry_proj4     +proj=longlat +datum=WGS84 +no_defs
Name: 0, d

In [10]:
type_by_name = {
    'DistribTransf_N': 'transformer',
    'DummyEquip': 'miscellaneous',
    'HVMVSubstation_N': 'substation',
    'Line_N': 'line',
    'Network_NEW_nodes': '',
    'NewConsumerGreenfield_N': 'meter',
    'SwitchingDevices_N': 'switch',
    'TransSubstation_N': 'substation',
    'Transformer_N': 'transformer',
    'VoltageRegulator_N': 'quality',
}

In [11]:
# Make node lookup dictionary
from collections import defaultdict
from invisibleroads_macros.disk import get_file_stem

excluded_prefixes = [
    'StreetMap_',
    'Network_',
    'Dummy',
    'Line',
]

def should_skip(path):
    source_stem = get_file_stem(source_path)
    for prefix in excluded_prefixes:
        if source_stem.startswith(prefix):
            return True
    return False
    
node_by_code = defaultdict(dict)

source_names = [
    'Network_NEW_nodes.shp',
    'DummyEquip.shp']
for source_name in source_names:
    source_path = join(source_folder, source_name)
    print(source_path)
    t = geotable.load(source_path)
    t.set_index(['Node'], inplace=True)
    type_name = type_by_name[get_file_stem(source_path)]
    for code, row in t.iterrows():
        d = dict(row)
        d['geometry_layer'] = type_name
        node_by_code[code].update(d)
        
for source_path in source_paths:
    if should_skip(source_path):
        continue
    t = geotable.load(source_path)
    t.set_index(['Code'], inplace=True)
    type_name = type_by_name[get_file_stem(source_path)]
    for code, row in t.iterrows():
        d = dict(row)
        d['geometry_layer'] = type_name
        node_by_code[code].update(d)

/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_nodes.shp
/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/DummyEquip.shp


In [12]:
len(node_by_code)

20453

In [13]:
# Make edge lookup dictionary
edge_by_code = defaultdict(dict)

t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Network_NEW_branches.shp')
t.set_index(['Node_A', 'Node_B'], inplace=True)
for code, row in t.iterrows():
    d = dict(row)
    d['geometry_layer'] = ''
    edge_by_code[code].update(d)
    
t = geotable.load('/tmp/greensboro-synthetic-network/GSO_RNM_GIS_Network/Rural/Line_N.shp')
t.set_index(['NodeA', 'NodeB'], inplace=True)
for code, row in t.iterrows():
    d = dict(row)
    d['geometry_layer'] = 'line'
    edge_by_code[code].update(d)

In [14]:
len(edge_by_code)

16392

In [15]:
# Add edges from branches
import networkx as nx
g = nx.Graph()
for node, d in node_by_code.items():
    g.add_node(node, **d)
for (node_a, node_b), d in edge_by_code.items():
    g.add_edge(node_a, node_b, **d)

In [16]:
len(g)

20453

In [18]:
nx.write_gpickle(g, 'example.pkl', -1)

In [20]:
ls -l

total 19420
-rw-rw-r--. 1 rhh rhh 19790309 Dec 26 15:51  example.pkl
-rw-rw-r--. 1 rhh rhh    45981 Dec 26 14:04 'Import Example Dataset 20181222.ipynb'
-rw-rw-r--. 1 rhh rhh    32983 Dec 26 15:51 'Import Example Dataset 20181226.ipynb'
-rw-rw-r--. 1 rhh rhh     1530 Dec 25 15:00  proj4s.txt
-rw-rw-r--. 1 rhh rhh     2403 Dec 26 14:33  save_geojson.py
